プログラムの流れ
1. 2000年からの履歴を取得
1. 最新のFX情報を取得して、Dataframeに保存
1. データフレームの結合
1. ファイルの保存

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import datetime
import pickle

In [2]:
start = datetime.datetime(2000, 1, 1)
end = datetime.datetime.now()

# 日経平均の取得
#### https://ntk-lab.com/import_stock_data/

In [3]:
def nikkei_download():
    # 日経平均
    df_nikkei=web.DataReader('^N225', 'yahoo',start,end)
    print ("配列長さ:",len(df_nikkei))
    df_nikkei=df_nikkei[['Open','Close']]
    df_nikkei = df_nikkei.reset_index()
    return df_nikkei

In [4]:
df_nikkei=nikkei_download()
display(df_nikkei.head(3))
display(df_nikkei.tail(3))

配列長さ: 5071


,Date,Open,Close
0,2000-01-04,18937.449219,19002.859375
1,2000-01-05,19003.509766,18542.550781
2,2000-01-06,18574.009766,18168.269531


,Date,Open,Close
5068,2020-09-09,22966.890625,23032.539062
5069,2020-09-10,23193.470703,23235.470703
5070,2020-09-11,23114.630859,23406.490234


# ダウ平均の取得
### https://jp.investing.com/indices/us-30-historical-data


In [5]:
#"df_dow

In [6]:
def dow_dayplus(row):
    y=row['Date']
    
    if (row['weekday']>=0 and row['weekday']<=3):
        # アメリカが月〜木曜日であれば、アメリカ時間に+1 して、日本時間にマージする
        #　例　6月16日（月）(USA)  => 6月17日（火）（jst）
        y=row['Date']+ datetime.timedelta(days=1)

    elif row['weekday']==4:
        # 金曜日であれば、   +3
        #　例　6月19日（金）(USA)  => 6月22日（月）（jst）

        y=row['Date']+ datetime.timedelta(days=3)
    
    return y
#df_dow['date2']=df_dow.apply(dow_dayplus,axis=1)
#df_dow.tail(30)

In [7]:
def dow_download():
    # ダウ平均
    df_dow = web.DataReader('^DJI', 'yahoo',start,end)
    
    df_dow['dow_compare']=df_dow['Close']/df_dow['Open']
    df_dow=df_dow[['dow_compare']]
    df_dow = df_dow.reset_index()
    
    print ("配列長さ:",len(df_dow))
    
    # ⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️ここでダウ平均の時間を＋する
    df_dow['weekday']=df_dow['Date'].apply(lambda x: x.weekday())
    df_dow['Date']=df_dow.apply(dow_dayplus,axis=1)

    #df_dow['Date2']=df_dow.applay(dow_download)
    #df_dow['Date']=df_dow['Date']+ datetime.timedelta(days=1)
    return df_dow
df_dow=dow_download()
df_dow.tail()

配列長さ: 5208


,Date,dow_compare,weekday
5203,2020-09-07,0.992670,4
5204,2020-09-09,0.984804,1
5205,2020-09-10,1.008255,2
5206,2020-09-11,0.982588,3
5207,2020-09-14,1.001875,4


# 結合

In [8]:
# 曜日の追加
#df['weekday']=df['Date'].apply(lambda x: x.weekday())

In [9]:
df=pd.merge(df_nikkei,df_dow,on=['Date'],how='inner')
df['Close']=df['Close'].astype('float')
df['Open']=df['Open'].astype('float')
df=df[['Date','Open','Close','dow_compare']]
df.info()
df.tail(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4899 entries, 0 to 4898
Data columns (total 4 columns):
Date           4899 non-null datetime64[ns]
Open           4899 non-null float64
Close          4899 non-null float64
dow_compare    4899 non-null float64
dtypes: datetime64[ns](1), float64(3)
memory usage: 191.4 KB


,Date,Open,Close,dow_compare
4889,2020-08-28,23232.300781,22882.650391,1.003812
4890,2020-08-31,23147.140625,23139.759766,1.001838
4891,2020-09-01,23089.630859,23138.070312,0.992543
4892,2020-09-02,23261.089844,23247.150391,1.007245
4893,2020-09-03,23524.490234,23465.529297,1.012657
4894,2020-09-04,23130.320312,23205.429688,0.972570
4895,2020-09-07,23145.470703,23089.949219,0.992670
4896,2020-09-09,22966.890625,23032.539062,0.984804
4897,2020-09-10,23193.470703,23235.470703,1.008255
4898,2020-09-11,23114.630859,23406.490234,0.982588


# ファイルの保存 

In [10]:
# ファイルの保存
df.to_pickle('data/stock.pkl')
df

,Date,Open,Close,dow_compare
0,2000-01-04,18937.449219,19002.859375,0.987451
1,2000-01-05,19003.509766,18542.550781,0.969002
2,2000-01-06,18574.009766,18168.269531,1.012128
3,2000-01-07,18194.050781,18193.410156,1.012588
4,2000-01-11,18246.099609,18850.919922,1.003444
...,...,...,...,...
4894,2020-09-04,23130.320312,23205.429688,0.972570
4895,2020-09-07,23145.470703,23089.949219,0.992670
4896,2020-09-09,22966.890625,23032.539062,0.984804
4897,2020-09-10,23193.470703,23235.470703,1.008255
